In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [3]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score

In [4]:
data = pd.read_csv("Bank.csv",delimiter=';')
df=data

In [5]:
df['log_age'] = np.log(df['age'])
df["Any_Loans"] = df["housing"]+df["loan"]
df["Any_Loans"].replace(["yesno","noyes","yesyes"],"yes",inplace=True)
df["Any_Loans"].replace(["nono",],"no",inplace=True)

In [6]:
#i am dropping multiple columns

dropped_labels = ['age','day_of_week','contact','pdays','housing','loan','poutcome','emp.var.rate','previous','cons.price.idx','duration','cons.conf.idx','euribor3m','nr.employed']
df.drop(dropped_labels,axis=1,inplace=True)

In [7]:
df.head(1)

,job,marital,education,default,month,campaign,y,log_age,Any_Loans
0,housemaid,married,basic.4y,no,may,1,no,4.025352,no


In [8]:
df['Month'] = df['month']
df['Month'] = df['Month'].replace(['jan','feb','mar'],'First_Quarter')
df['Month'] = df['Month'].replace(['apr','may','jun'],'Second_Quarter')
df['Month'] = df['Month'].replace(['jul','aug','sep'],'Third_Quarter')
df['Month'] = df['Month'].replace(['oct','nov','dec'],'Fourth_Quarter')
df.drop('month',axis=1,inplace=True)

In [9]:
# I am merging some classes together
df['education'] = df['education'].replace(['basic.4y','basic.6y','basic.9y'],'elementary')
df['y'] = df['y'].replace(['no'],0)
df['y'] = df['y'].replace(['yes'],1)

In [10]:
df.head(2)

,job,marital,education,default,campaign,y,log_age,Any_Loans,Month
0,housemaid,married,elementary,no,1,0,4.025352,no,Second_Quarter
1,services,married,high.school,unknown,1,0,4.043051,no,Second_Quarter


In [11]:
df["education"].value_counts()

elementary             12513
university.degree      12168
high.school             9515
professional.course     5243
unknown                 1731
illiterate                18
Name: education, dtype: int64

In [12]:
df["job"].value_counts()

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [13]:
print(msn)

NameError: name 'msn' is not defined

In [ ]:
#df['campaign'].plot.hist()

In [14]:
for i in range(len(df['campaign'])):
    if (df['campaign'][i]>=1 and df['campaign'][i]<=5):
        df['campaign'][i]="Significance"
    
    elif (df['campaign'][i]>5 and df['campaign'][i]<=10):
        df['campaign'][i]="Frequent"
        
    else:
        df['campaign'][i]="Too Frequent"
#     if calls>=1 and calls<=5 :
#         df['campaign'].replace(calls,'Significance')
#     elif calls>5 and calls<=10 :
#         df['campaign'].replace(calls,'Frequent')
#     else:
#         df['campaign'].replace(calls,'Too Frequent')

C:\Users\MAHES\AppData\Local\Temp\ipykernel_19700\2768168671.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['campaign'][i]="Significance"


In [15]:
df.rename(columns={'campaign':"FrequencyOfPrvCalls"},inplace=True)

In [16]:
df.head(1)

,job,marital,education,default,FrequencyOfPrvCalls,y,log_age,Any_Loans,Month
0,housemaid,married,elementary,no,Significance,0,4.025352,no,Second_Quarter


In [ ]:
print(Ar)

In [17]:
x = df.drop('y',axis=1)
y = df['y']

In [18]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.10,random_state=11,stratify=y)

In [19]:
from sklearn.model_selection import train_test_split as tts
xtt,x_val,ytt,y_val = tts(x_train,y_train,test_size=0.20,random_state=6,stratify=y_train)

In [20]:
x_train.head(1)

,job,marital,education,default,FrequencyOfPrvCalls,log_age,Any_Loans,Month
324,technician,married,professional.course,no,Significance,3.988984,yes,Second_Quarter


In [21]:
step1 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse=False,handle_unknown='ignore'),[0,1,2,3,4,6,7])
],remainder='passthrough')

In [22]:
step2 = GaussianNB()

In [23]:
pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

In [24]:
pipe.fit(xtt,ytt)
#cv = cross_val_score(pipe,x_train,y_train,cv=5,scoring='recall',)
#check recall for training
y_pred_train = pipe.predict(xtt)
print("training recall:",recall_score(ytt,y_pred_train))
y_pred_val = pipe.predict(x_val)
print('val racallL',recall_score(y_val,y_pred_val))
y_test_pred = pipe.predict(x_test)
print("test recall:",recall_score(y_test,y_test_pred))
#print("recall:",recall_score(y_train,y_pred))
#print("cv recall,train",cv.mean())
print()


training recall: 0.8718946423226579
val racallL 0.8766467065868263
test recall: 0.8663793103448276



C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
print("cv recall training",cross_val_score(pipe,xtt,ytt,cv=5,scoring='recall').mean())
print("cv recall validation",cross_val_score(pipe,x_val,y_val,cv=5,scoring='recall').mean())
print("cv recall test",cross_val_score(pipe,x_test,y_test,cv=5,scoring='recall').mean())

C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

cv recall training 0.8721932815982385
cv recall validation 0.9353293413173652
cv recall test 0.8792426367461431


C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

In [27]:
from xgboost import XGBClassifier
step2_xgb = XGBClassifier()

In [28]:
pipe_new = Pipeline([
    ('step1',step1),
    ('step2',step2_xgb)
])

In [29]:
pipe_new.fit(xtt,ytt)
cv = cross_val_score(pipe_new,xtt,ytt,cv=5,scoring='recall',)
#check recall for training
y_pred_train = pipe_new.predict(xtt)
print("training recall:",recall_score(ytt,y_pred_train))
y_pred_val = pipe_new.predict(x_val)
print('val racallL',recall_score(y_val,y_pred_val))
y_test_pred = pipe_new.predict(x_test)
print("test recall:",recall_score(y_test,y_test_pred))

C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

training recall: 0.14606405267883868
val racallL 0.0658682634730539
test recall: 0.09051724137931035


In [30]:
print("cv recall training",cross_val_score(pipe_new,xtt,ytt,cv=5,scoring='recall').mean())
print("cv recall validation",cross_val_score(pipe_new,x_val,y_val,cv=5,scoring='recall').mean())
print("cv recall test",cross_val_score(pipe_new,x_test,y_test,cv=5,scoring='recall').mean())

C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

cv recall training 0.08829650116806745


C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

cv recall validation 0.08862275449101796


C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\MAHES\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

cv recall test 0.09270687237026647
